In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pearsonr
import pandas as pd

import sys
sys.path.append('../../src/')
from coal_cov import *
from seg_sites_covar import CorrSegSites
from plot_utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.autolayout'] = True
plt.rcParams['pdf.fonttype'] = 3
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

import os
main_figdir = '../../plots/two_locus_stats/'
supp_figdir = '../../plots/supp_figs/two_locus_stats/'
os.makedirs(main_figdir, exist_ok=True)
os.makedirs(supp_figdir, exist_ok=True)

### Simulation under a model of exponential growth 

In [3]:
# NOTE : growth always starts ~ 1000 Generations in the past ... 
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-3, 1e-2, 5e-2]
rec_rate = 1e-3


total_sims = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialGrowth(ta=t, r=r, T=500, reps=50000, rec_rate=rec_rate)
        ts_reps = cur_sim._simulate()
        cur_sim._two_locus_branch_length(ts_reps)
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corr_LALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims.append([t,r, mu_LA, var_LA, cov_LALB, corr_LALB])
            
total_sims = np.vstack(total_sims)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:05<00:00, 61.68s/it]


In [4]:
growth_df = pd.DataFrame(total_sims)
growth_df.columns = ['$t_a$', '$r$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A, L_B)$']
growth_df.to_csv('../../results/two_loci/growth_model.csv', index=False)
growth_df

,$t_a$,$r$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A, L_B)$"
0,0.0,0.001,6087.518173,4.691786e+06,779500.950249,0.166678
1,0.0,0.010,1945.178045,6.458962e+04,19441.958582,0.300370
2,0.0,0.050,1252.998297,2.661182e+03,1016.081143,0.383271
3,100.0,0.001,6004.204914,4.622373e+06,798146.271779,0.173105
4,100.0,0.010,1843.873281,6.494130e+04,20864.018436,0.322482
5,100.0,0.050,1153.218551,2.621729e+03,1073.400377,0.409480
6,500.0,0.001,5685.105767,4.189090e+06,627472.823868,0.150182
7,500.0,0.010,1449.309935,5.914346e+04,25321.454950,0.427168
8,500.0,0.050,753.403538,2.561307e+03,1475.120191,0.574928


### Simulation under a bottleneck model

In [5]:
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-1, 1e-2, 1e-3]
rec_rate = 1e-3

total_sims_bot = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialBottleneck(Ne=1e4, ta=t, Tstart=50, Tend=500, Nbot=r*1e4, reps=50000, rec_rate=rec_rate)
        ts_reps = cur_sim._simulate()
        cur_sim._two_locus_branch_length(ts_reps)
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corrLALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims_bot.append([t, r, mu_LA, var_LA, cov_LALB, corrLALB])
        
        
total_sims_bot = np.vstack(total_sims_bot)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:08<00:00, 82.98s/it]


In [6]:
bottleneck_df = pd.DataFrame(total_sims_bot)
bottleneck_df.columns = ['$t_a$', '$\phi$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A,L_B)$']
bottleneck_df.to_csv('../../results/two_loci/bottleneck.csv', index=False)
bottleneck_df

,$t_a$,$\phi$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A,L_B)$"
0,0.0,0.100,32705.245938,1.543057e+09,1.889688e+08,0.122052
1,0.0,0.010,4677.130341,3.238840e+08,1.035026e+08,0.311465
2,0.0,0.001,139.687120,1.600915e+03,1.462250e+03,0.912142
3,100.0,0.100,33455.467909,1.551079e+09,1.808965e+08,0.117548
4,100.0,0.010,5832.928726,4.073364e+08,1.314773e+08,0.326441
5,100.0,0.001,139.949182,1.608784e+03,1.544450e+03,0.959516
6,500.0,0.100,40004.922338,1.566143e+09,2.915162e+07,0.018472
7,500.0,0.010,40449.487997,1.593508e+09,3.151265e+07,0.019701
8,500.0,0.001,40378.871050,1.614716e+09,4.228108e+07,0.026353
